https://jimkang.com/quadtreevis/

In [55]:
"""
I think, the best way to depict Quad Tree would be as a map with coordinates (x, y), so I'll try to replicate that.

Also I know this implementation sucks without the checks and all that, but I don't want to spend more time on it.
"""

from typing import Iterator, Optional


MAXIMUM_DATA_POINTS = 4  # the amount of data points one node can hold before splitting


class Node:
    """
    aka QuadTree

    Every node represents a section of an imaginary "map", so it has `x` and `y` coordinates bounds.
    Each one can contain either nodes or be a leaf with concrete data points.
    """
    
    def __init__(self, x1: int, x2: int, y1: int, y2: int) -> None:
        """
            (x1, y2)                 (x2, y2)
                    x---------------x
                    |               |
                    |               |
                    |               |
                    |               |
                    |               |
                    x---------------x
            (x1, y1)                 (x2, y1)
        """
        
        self._x1, self._x2, self._y1, self._y2 = x1, x2, y1, y2
        self._data_points: list[tuple[int, int]] = []  # point in a format (x, y)
        
        # whether a node is a leaf OR it has any number of children is a contravariant
        self._top_left: Optional['Node'] = None
        self._bottom_left: Optional['Node'] = None
        self._bottom_right: Optional['Node'] = None
        self._top_right: Optional['Node'] = None
    
    @property
    def _is_leaf(self) -> bool:
        return not (self._top_left or self._bottom_left or self._bottom_right or self._top_right)
    
    def _accepts(self, x: int, y: int) -> bool:
        """Check if the coordinate belongs to this node."""
        
        return (
            self._x1 <= x <= self._x2
            and self._y1 <= y <= self._y2
        )
    
    @property
    def _nodes(self) -> Iterator['Node']:
        for node in (
            self._top_left,
            self._bottom_left,
            self._bottom_right,
            self._top_right,
        ):
            yield node

    def add_coordinate(self, x: int, y: int) -> None:
        """Add a data point and rebalance tree if necessary."""
        
        if not self._is_leaf:
            for node in self._nodes:
                if node._accepts(x, y):
                    node.add_coordinate(x, y)
                    return
            assert False  # this should never be called
        
        if len(self._data_points) < MAXIMUM_DATA_POINTS:
            self._data_points.append((x, y))
            return
        
        # we can optimize by splitting it not in 4, but partially; I will just split in 4 for simplicity
        # damn, this is so error-prone
        self._top_left = Node(self._x1, self._x2 // 2, self._y2 // 2 + 1, self._y2)
        self._bottom_left = Node(self._x1, self._x2 // 2, self._y1, self._y2 // 2)
        self._bottom_right = Node(self._x2 // 2 + 1, self._x2, self._y1, self._y2 // 2)
        self._top_right = Node(self._x2 // 2 + 1, self._x2, self._y2 // 2 + 1, self._y2)

        for (ex, ey) in self._data_points + [(x, y)]:
            for node in self._nodes:
                if node._accepts(ex, ey):
                    node.add_coordinate(ex, ey)
                    break
        self._data_points = []

    def remove_coordinate(self) -> None:
        """Maybe I'll add it later."""
        raise NotImplemented()
    
    def coordinate_exists(self, x: int, y: int) -> bool:
        """Simple check whether our tree has this coordinate added."""
        
        if self._is_leaf:
            for data_point in self._data_points:
                if (x, y) == data_point:
                    return True
            return False
        
        for node in self._nodes:
            if node._accepts(x, y):
                return node.coordinate_exists(x, y)
        return False
    
    def __repr__(self) -> str:
        base = f"Node({self._x1, self._x2, self._y1, self._y2}"
        if self._is_leaf:
            return base + f", data={str(self._data_points)})\n"
        base += ")\n"
        child_nodes = ""
        for node in self._nodes:
            child_nodes += str(node)
        return base + child_nodes

class QuadTree(Node):
    """
    As quad tree is a composite data structure, it can be represented with a `Node` instance and then branch into smaller
    sub-trees via nodes as well.
    The purpose of this class is to provide a facade for a tree, because client doesn't have to know about the `Node`, only
    about the tree and data it stores. Having said that, maybe it should be named `_Node`, but that's just some code for
    learning a concept so who cares really.
    Have a nice day.
    """

    def add_coordinate(self, x, y):
        if not self._accepts(x, y):
            assert False  # only coordinates in a predetermined range are allowed
        return super().add_coordinate(x, y)

In [56]:
qt = QuadTree(0, 100, 0, 100)
qt.add_coordinate(100, 100)
qt.add_coordinate(34, 56)
qt.add_coordinate(68, 54)
# bottom left
qt.add_coordinate(1, 1)
qt.add_coordinate(4, 5)
qt.add_coordinate(10, 10)
qt.add_coordinate(20, 20)
qt.add_coordinate(30, 30)

In [60]:
qt.coordinate_exists(68, 54)

True

In [61]:
qt.coordinate_exists(68, 53)

False

In [58]:
qt

Node((0, 100, 0, 100))
Node((0, 50, 51, 100), data=[(34, 56)])
Node((0, 50, 0, 50))
Node((0, 25, 26, 50), data=[])
Node((0, 25, 0, 25), data=[(1, 1), (4, 5), (10, 10), (20, 20)])
Node((26, 50, 0, 25), data=[])
Node((26, 50, 26, 50), data=[(30, 30)])
Node((51, 100, 0, 50), data=[])
Node((51, 100, 51, 100), data=[(100, 100), (68, 54)])